# Solving an electric circuit using Particle Swarm Optimization
## Introduction

PSO can be utilized in a wide variety of fields. In this example, the problem consists of analysing a given electric circuit and finding the electric current that flows through it. To accomplish this, the pyswarms library will be used to solve a non-linear equation by restructuring it as an optimization problem. The circuit is composed by a source, a resistor and a diode, as shown below.

![Circuit](https://user-images.githubusercontent.com/39431903/43938822-29aaf9b8-9c66-11e8-8e54-01530db005c6.png)


### Mathematical Formulation

Kirchhoff's voltage law states that the directed sum of the voltages around any closed loop is zero. In other words, the sum of the voltages of the passive elements must be equal to the sum of the voltages of the active elements, as expressed by the following equation:

$ U = v_D + v_R $, where $U$ represents the voltage of the source and, $v_D$ and $v_R$ represent the voltage of the diode and the resistor, respectively.

To determine the current flowing through the circuit, $v_D$ and $v_R$ need to be defined as functions of $I$. A simplified Shockley equation will be used to formulate the current-voltage characteristic function of the diode. This function relates the current that flows through the diode with the voltage across it. Both $I_s$ and $v_T$ are known properties.

$I = I_s  e^{\frac{v_D}{v_T}}$

Where:

- $I$ : diode current
- $I_s$ : reverse bias saturation current
- $v_D$ : diode voltage
- $v_T$ : thermal voltage

Which can be formulated over $v_D$:

$v_D = v_T \log{\left |\frac{I}{I_s}\right |}$

The voltage over the resistor can be written as a function of the resistor's resistance $R$ and the current $I$:

$v_R = R  I$

And by replacing these expressions on the Kirschhoff's voltage law equation, the following equation is obtained:

$ U = v_T \log{\left |\frac{I}{I_s}\right |} + R I $


To find the solution of the problem, the previous equation needs to be solved for $I$, which is the same as finding $I$ such that the cost function $c$ equals zero, as shown below. By doing this, solving for $I$ is restructured as a minimization problem. The absolute value is necessary because we don't want to obtain negative currents.

$c = \left | U - v_T \log{\left | \frac{I}{I_s} \right |} - RI \right |$



### Known parameter values

The voltage of the source is $ 10 \space V $ and the resistance of the resistor is $ 100 \space \Omega $. The diode is a silicon diode and it is assumed to be at room temperature.

$U = 10 \space V $

$R = 100 \space \Omega $

$I_s = 9.4 \space pA = 9.4 \times 10^{-12} \space A$ (reverse bias saturation current of silicon diodes at room temperature, $T=300 \space K$)

$v_T = 25.85 \space mV = 25.85 \times 10^{-3} \space V $ (thermal voltage at room temperature, $T=300 \space K$)

## Optimization

In [2]:
# Import modules
import sys
import numpy as np

# Import PySwarms
import pyswarms as ps

In [3]:
print('Running on Python version: {}'.format(sys.version))

Running on Python version: 3.7.1 (v3.7.1:260ec2c36a, Oct 20 2018, 14:05:16) [MSC v.1915 32 bit (Intel)]


### Defining the cost fuction

The first argument of the cost function is a *numpy.ndarray*. Each dimension of this array represents an unknown variable. In this problem, the unknown variable is just $I$, thus the first argument is a unidimensional array. As default, the thermal voltage is assumed to be $25.85 \space mV$.

In [4]:
def cost_function(I, U, R, I_s, v_t = 25.85 / 1000):
    
    c = abs(U - v_t * np.log(abs(I[:, 0] / I_s)) - R * I[:, 0])
    
    return c

### Setting the optimizer

To solve this problem, the global-best optimizer is going to be used. 

In [5]:
%%time
# Set-up hyperparameters
options = {'c1': 0.5, 'c2': 0.3, 'w':0.9}

# Call instance of PSO
optimizer = ps.single.GlobalBestPSO(n_particles=10, dimensions=1, options=options)

# Perform optimization
cost, pos = optimizer.optimize(cost_function, iters=1000, U=10, R=100, I_s=9.4 * (10 ** -12))

2019-02-11 22:35:06,506 - pyswarms.single.global_best - INFO - Optimize for 1000 iters with {'c1': 0.5, 'c2': 0.3, 'w': 0.9}
pyswarms.single.global_best: 100%|██████████████████████████████████████████████████████████████|1000/1000, best_cost=0
2019-02-11 22:35:25,195 - pyswarms.single.global_best - INFO - Optimization finished | best cost: 0.0, best pos: [0.09404769]


Wall time: 18.7 s


In [10]:
print(pos[0])

0.09404768643017938


In [8]:
print(cost)

0.0


### Solution

The current flowing through the circuit is approximately $ 0.094 \space A$ and the cost is equal to zero.